In [1]:
from bs4 import BeautifulSoup
import requests
import re

In [2]:
# assign variables destination
url = "https://www.seek.com.au/data-analyst-jobs/in-All-Sydney-NSW/full-time?daterange=1&salaryrange=80000-200000&salarytype=annual&sortmode=ListedDate"

# Grab the details/ content of that url
req = requests.get(url)


In [3]:
soup = BeautifulSoup(req.text, 'html.parser')
# Parsers convert the input into single entities known as tokens and further convert the tokens into a graph
# or a tree structure for processing

# Job titles

In [4]:
job_titles = []
levels = soup.find_all('article', {'class' : '_37iADb_'})
for level in levels:
    divs = level.find_all('a', {'class' : '_2S5REPk'})
    for div in divs:
        job_titles.append(div.text)
print(job_titles)

['SQL Developer', 'Pricing Analyst', 'SQL Server BI/DW Developer', 'Data Analyst', 'Privacy Risk Analyst', 'Manager Business Reporting, Clerk Grade 11/12 - 215198', 'Manager Business Reporting, Clerk Grade 11/12 - 215198', 'Technical Business Analyst', 'Finance Coordinator', 'Compliance Data Analyst - Chatswood', 'Performance Engineer - Python, Quant C++, Network (UDP) ,FPGA, Pricing Data, HFT', 'FPGA Software Eng (C++, Python) Low Latency Trading -AU PR + Relocation Provided', 'IT Support Officer', 'Head of Acquisition and Pricing - Mortgages', 'Data Engineer - Grow your career!', 'Lead Financial Planning & Analyst - APAC', 'Acquisition / Lead Generation Expert', 'Digital Product Manager (B2B)', 'Paid Media Specialist']


# Hirer

In [5]:
hirer = []
# levels = soup.find_all('article', {'class' : '_37iADb_'})
for level in levels:
    hirer_divs = level.find_all('span', {'class' : '_3mgsa7- _15GBVuT _2Ryjovs'})
    for hirer_div in hirer_divs:
        hirer.append(hirer_div.text)

desiredwords = ['at ']
hirer = [x for x in hirer if any(word in x for word in desiredwords)]
print(hirer)

['at u&u. Recruitment Partners', "at O'Brien Glass Industries Limited", 'at Mane Consulting', 'at Webcentral Group', 'at Rabobank', 'at Department of Education', 'at Department of Education', 'at The Recruitment Alternative: Affordable Recruitment for Small Businesses', 'at TransGrid', 'at Holcim', 'at Westbury Partners', 'at Westbury Partners', 'at CEA', 'at ING', 'at The Recruitment Company', 'at SALT SEARCH PTY LTD', 'at Private Advertiser', 'at Solar Analytics', 'at The-Lowdown Agency']


# Job ids

In [15]:
mylist = []
job_ids_final = soup.find_all(attrs={"data-job-id": re.compile("^[0-9]{8}$")})
for jif in job_ids_final:
    mylist.append(jif["data-job-id"])

# Build URLs

In [16]:
# Put Python loop output in a list
# https://stackoverflow.com/questions/39322625/how-to-put-python-loop-output-in-a-list/39323039
base_url = 'https://www.seek.com.au/job/'
dest_url = []
for myl in mylist:
    dest_url.append(base_url + myl)

# Scrape job details

In [17]:
#dest_url
jd1 = []
for dtl in dest_url:
    requestcheck = requests.get(dtl)
    parser = BeautifulSoup(requestcheck.text, 'html.parser')
    jd2 = parser.find_all('div', {'class' : 'FYwKg PrHFr _1EtT-_4'})
    for j in jd2:
        jd1.append(j.text)
#print(jd1)

In [18]:
# splitting a list above into sublists after 'Full time' text
jd3 = [i.strip().split(' ') for i in ' '.join(jd1).split('Full Time') if len(i) > 0 ]
#print(jd3)

In [19]:
# function to convert / combine list into string
from itertools import chain

def collapse(lst):
    return ' ' .join(chain.from_iterable(lst))

In [20]:
# Push list through the function to convert to string
out = collapse(jd3)
#print(out)

In [21]:
# Converting string back to list after the text 'Sydney'
jobdetails = [i.strip() for i in out[6:].split('Sydney') if not i.isspace()]


# Putting it all together

In [22]:
for one, two, three, four in zip(job_titles, hirer, jobdetails, dest_url):
    final = print(one, two, "|| Details: " + three + "|| Learn more at: " + four)

SQL Developer at u&u. Recruitment Partners || Details: CBD, Inner West & Eastern Suburbs Information & Communication Technology Engineering - Software Up to $125,000 - 130,000 Package|| Learn more at: https://www.seek.com.au/job/52617308
Pricing Analyst at O'Brien Glass Industries Limited || Details: Accounting Analysis & Reporting|| Learn more at: https://www.seek.com.au/job/52617290
SQL Server BI/DW Developer at Mane Consulting || Details: Information & Communication Technology Database Development & Administration|| Learn more at: https://www.seek.com.au/job/52617209
Data Analyst at Webcentral Group || Details: CBD, Inner West & Eastern Suburbs Information & Communication Technology Business/Systems Analysts|| Learn more at: https://www.seek.com.au/job/52617138
Privacy Risk Analyst at Rabobank || Details: CBD, Inner West & Eastern Suburbs Banking & Financial Services Compliance & Risk|| Learn more at: https://www.seek.com.au/job/52616942
Manager Business Reporting, Clerk Grade 11/12

In [25]:
# https://nbconvert.readthedocs.io/en/latest/removing_cells.html
# Removing cell in html

from traitlets.config import Config
import nbformat as nbf
from nbconvert.exporters import HTMLExporter
from nbconvert.preprocessors import TagRemovePreprocessor

# Setup config
c = Config()

# Configure tag removal - be sure to tag your cells to remove  using the
# words remove_cell to remove cells. You can also modify the code to use
# a different tag word
c.TagRemovePreprocessor.remove_cell_tags = ("remove_cell",)
c.TagRemovePreprocessor.remove_all_outputs_tags = ('remove_output',)
c.TagRemovePreprocessor.remove_input_tags = ('remove_input',)
c.TagRemovePreprocessor.enabled = True

# Configure and run out exporter
c.HTMLExporter.preprocessors = ["nbconvert.preprocessors.TagRemovePreprocessor"]

exporter = HTMLExporter(config=c)
exporter.register_preprocessor(TagRemovePreprocessor(config=c),True)

# Configure and run our exporter - returns a tuple - first element with html,
# second with notebook metadata
output = HTMLExporter(config=c).from_filename("webpage_scrape.ipynb")

# Write to output html file
with open("outputtoemail.html",  "w", encoding='utf-8') as f:
    f.write(output[0])